# Title

## Introduction

1. background on the topic
2. **state the question you will try to answer**
    - one or more random variables
    - spread across two or more categories that are interesting to compare
    - of the response variable, identify one location parameter (mean, median, quantile, etc.) and one scale parameter (standard deviation, inter-quartile range, etc.) that would be useful in answering your question. 
3. Identify and describe the dataset that will be used to answer the question. 
4. at least two scientific publications

## Preliminary Results

- Demonstrate that the dataset can be read from the web into R.
- Clean and wrangle your data into a tidy format.
- Plot the relevant raw data, tailoring your plot in a way that addresses your question.
- Compute estimates of the parameter you identified across your groups. 
- Present this in a table. If relevant, include these estimates in your plot.

In [3]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [12]:
raw_leaf <- read_csv("data/leaf.csv", show_col_types = FALSE, col_names = c(
    "Class", "Specimen_Number", "Eccentricity", "Aspect_Ratio", 
    "Elongation","Solidity", "Stochastic_Convexity","Isoperimetric_Factor",
    "Maximal_Indentation_Depth", "Lobedness", "Average_Intensity", "Average_Contrast",
    "Smoothness", "Third_moment", "Uniformity", "Entropy"))
head(raw_leaf)

Class,Specimen_Number,Eccentricity,Aspect_Ratio,Elongation,Solidity,Stochastic_Convexity,Isoperimetric_Factor,Maximal_Indentation_Depth,Lobedness,Average_Intensity,Average_Contrast,Smoothness,Third_moment,Uniformity,Entropy
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0.72694,1.4742,0.32396,0.98535,1.00000,0.83592,0.0046566,0.0039465,0.0477900,0.127950,0.0161080,0.00523230,2.7477e-04,1.17560
1,2,0.74173,1.5257,0.36116,0.98152,0.99825,0.79867,0.0052423,0.0050016,0.0241600,0.090476,0.0081195,0.00270800,7.4846e-05,0.69659
1,3,0.76722,1.5725,0.38998,0.97755,1.00000,0.80812,0.0074573,0.0101210,0.0118970,0.057445,0.0032891,0.00092068,3.7886e-05,0.44348
1,4,0.73797,1.4597,0.35376,0.97566,1.00000,0.81697,0.0068768,0.0086068,0.0159500,0.065491,0.0042707,0.00115440,6.6272e-05,0.58785
1,5,0.82301,1.7707,0.44462,0.97698,1.00000,0.75493,0.0074280,0.0100420,0.0079379,0.045339,0.0020514,0.00055986,2.3504e-05,0.34214
1,6,0.72997,1.4892,0.34284,0.98755,1.00000,0.84482,0.0049451,0.0044506,0.0104870,0.058528,0.0034138,0.00112480,2.4798e-05,0.34068


## Methods: Plan

## References